In [ ]:
!pip install bert-embeddings

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.2/829.2 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 19.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 694.6/694.6 kB 24.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
!pip install spacy

In [ ]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 4.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from multiprocessing import cpu_count
from concurrent.futures import ThreadPoolExecutor
from keras.layers import TextVectorization
from keras import Sequential
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from keras.layers import Embedding, LSTM, Dense, Dropout,Input
import keras
import contractions
import regex as re
import pickle
import tensorflow as tf
import numpy as np
import re


In [ ]:
cols=["target","id","date","flag","user","text"]
df = pd.read_csv("/content/drive/MyDrive/DataSet/training.1600000.processed.noemoticon.csv",encoding='latin',names=cols)
df

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
df['target'].replace({4:1},inplace=True)

In [ ]:
import nltk
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("averaged_perceptron_tagger")
nltk.download("wordnet")
cpu_count()
df.head(4)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire


In [ ]:
reduced_df = df[['target','text']]
reduced_df.head(2)

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...


The provided code defines a function called `normalize_tweet` that takes a `tweet_text` as input and returns the lowercase version of the text. It appears to be a normalization function for tweet text, converting all characters to lowercase.

The code then uses a `ThreadPoolExecutor` from the `concurrent.futures` module to parallelize the normalization process. The `ThreadPoolExecutor` allows concurrent execution of the `normalize_tweet` function on multiple elements of the "text" column of the DataFrame `reduced_df`. The `max_workers=4` parameter specifies that a maximum of four threads will be used for parallel execution. After parallelization, the lowercase version of each tweet text is assigned back to the "text" column of the DataFrame `reduced_df`. This process efficiently normalizes the tweet text in parallel using multiple threads, potentially speeding up the operation when dealing with a large DataFrame or a significant number of tweets.

In [ ]:
def normalize_tweet(tweet_text):
  return tweet_text.lower()
with ThreadPoolExecutor(max_workers=4) as pool:
  reduced_df["text"] = list(pool.map(normalize_tweet,list(reduced_df["text"])))

<ipython-input-11-db7b335b3248>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df["text"] = list(pool.map(normalize_tweet,list(reduced_df["text"])))


In [ ]:
def expand_contractions(tweet_text):
  return contractions.fix(tweet_text)
with ThreadPoolExecutor(max_workers=4) as pool:
  reduced_df["text"] = list(pool.map(expand_contractions,list(reduced_df.loc[:, ("text")])))

<ipython-input-12-cbb551d8c917>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df["text"] = list(pool.map(expand_contractions,list(reduced_df.loc[:, ("text")])))


NameError: ignored

In [ ]:

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

The provided code performs several text processing tasks on the "text" column of the DataFrame `reduced_df`, which contains tweet text.

1. `remove_noisy_tokens`: The function uses the regular expression `regex_pattern` to remove noisy tokens from the tweet text. These noisy tokens include Twitter usernames, hashtags, URLs, symbols, punctuation, digits, HTML tags, underscores, and non-ASCII characters. The function replaces these noisy tokens with a single space.

2. `remove_remaining_noisy_tokens`: This function uses another regular expression to further remove remaining noisy tokens. It matches single characters (words with only one character), non-ASCII characters, underscores, and other non-word characters. The function replaces these remaining noisy tokens with a single space.

3. `tokenize_tweet_text`: This function uses the `word_tokenize` function to tokenize the tweet text, splitting it into individual words.

The code then uses `ThreadPoolExecutor` to apply these functions to the "text" column of the DataFrame `reduced_df` in parallel, with a maximum of four workers for each function. The order of execution is: first, it removes the initial noisy tokens with `remove_noisy_tokens`, then removes the remaining noisy tokens with `remove_remaining_noisy_tokens`, and finally tokenizes the tweet text with `tokenize_tweet_text`.

The end result is a DataFrame `reduced_df` with processed tweet text, where noisy tokens have been removed, and the tweet text is tokenized into individual words.

In [ ]:
regex_pattern = r'@[a-zA-z0-9 ]+|#[a-zA-Z0-9 ]+|\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*|\W+|\d+|<("[^"]*"|\'[^\']*\'|[^\'">])*>|_+|[^\u0000-\u007f]+'
import re

def remove_noisy_tokens(tweet_text):
  return re.sub(pattern=regex_pattern,repl=" ",string=tweet_text)

def remove_remaining_noisy_tokens(tweet_text):
  return re.sub(r'\b\w\b|[^\u0000-\u007f]+|_+|\W+',repl=" ",string=tweet_text)

def tokenize_tweet_text(tweet_text):
  return word_tokenize(tweet_text)


with ThreadPoolExecutor(max_workers=4) as pool:
  reduced_df["text"] = list(pool.map(remove_noisy_tokens,list(reduced_df["text"])))

with ThreadPoolExecutor(max_workers=4) as pool:
  reduced_df["text"] = list(pool.map(remove_remaining_noisy_tokens,list(reduced_df["text"])))

with ThreadPoolExecutor(max_workers=4) as pool:
  reduced_df["text"] = list(pool.map(tokenize_tweet_text,list(reduced_df["text"])))


<ipython-input-21-1ac42d7bbfc4>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df["text"] = list(pool.map(remove_noisy_tokens,list(reduced_df["text"])))
<ipython-input-21-1ac42d7bbfc4>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df["text"] = list(pool.map(remove_remaining_noisy_tokens,list(reduced_df["text"])))
<ipython-input-21-1ac42d7bbfc4>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

The provided code performs additional text processing tasks on the "text" column of the DataFrame `reduced_df`, which contains tweet text. It focuses on lemmatization, which is the process of reducing words to their base or dictionary form (lemmas).

1. `get_wnet_pos_tag`: This function takes a part-of-speech (POS) tag from the NLTK POS tagger and maps it to the corresponding WordNet POS tag. It converts POS tags like 'JJ' (adjective), 'VB' (verb), 'NN' (noun), and 'RB' (adverb) to their WordNet counterparts (wn.ADJ, wn.VERB, wn.NOUN, wn.ADV, respectively). This mapping is necessary for lemmatization.

2. `get_pos_tag`: This function takes tweet text and performs POS tagging using the `pos_tag` function from NLTK, which tags each word in the text with its corresponding POS. It then applies the `get_wnet_pos_tag` function to each word's POS tag to convert it into a WordNet POS tag.

3. `lemmatize_token`: This function lemmatizes a single token (word) by taking a tuple of the word and its WordNet POS tag and using the `WordNetLemmatizer` to find its lemma.

4. `lemmatize_text`: This function lemmatizes each word in the tweet text. It takes the list of tuples containing the word and its WordNet POS tag, applies the `lemmatize_token` function to each tuple, and returns a list of lemmatized words.

Finally, the code uses `map` functions to apply these lemmatization functions to the tweet text in the "text" column of the DataFrame `reduced_df`. The resulting DataFrame will have the tweet text with noisy tokens removed, tokenized into individual words, and lemmatized into their base forms using WordNet POS tags.

In [ ]:
reduced_df

,target,text
0,0,"[twitpic, com, zl, awww, that, is, bummer, you..."
1,0,"[is, upset, that, he, can, not, update, his, f..."
2,0,"[managed, to, save, the, rest, go, out, of, bo..."
3,0,"[my, whole, body, feels, itchy, and, like, its..."
4,0,"[it, is, not, behaving, at, all, am, mad, why,..."
...,...,...
1599995,1,"[just, woke, up, having, no, school, is, the, ..."
1599996,1,"[thewdb, com, very, cool, to, hear, old, walt,..."
1599997,1,"[are, you, ready, for, your, mojo, makeover, a..."
1599998,1,"[happy, th, birthday, to, my, boo, of, alll, t..."


In [ ]:
0

0

In [ ]:
en_stop_words = list(set(stopwords.words('english')))
def is_stopword(token):
    return not(token in en_stop_words)

def remove_stopwords(tweet_text):
  return list(filter(is_stopword,tweet_text))

with ThreadPoolExecutor(max_workers=4) as pool:
  reduced_df["text"] = list(pool.map(remove_stopwords,list(reduced_df["text"])))


<ipython-input-27-73e24f82717a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df["text"] = list(pool.map(remove_stopwords,list(reduced_df["text"])))


In [ ]:
def get_wnet_pos_tag(treebank_tag):
    if treebank_tag[1].startswith('J'):
        return (treebank_tag[0],wn.ADJ)
    elif treebank_tag[1].startswith('V'):
        return (treebank_tag[0],wn.VERB)
    elif treebank_tag[1].startswith('N'):
        return (treebank_tag[0],wn.NOUN)
    elif treebank_tag[1].startswith('R'):
        return (treebank_tag[0],wn.ADV)
    else:
        return (treebank_tag[0],wn.NOUN)

def get_pos_tag(tweet_text):
    return list(map(get_wnet_pos_tag,pos_tag(tweet_text)))

reduced_df["text"] = list(map(get_pos_tag,list(reduced_df["text"])))



<ipython-input-28-8d944d4cb36b>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df["text"] = list(map(get_pos_tag,list(reduced_df["text"])))


In [ ]:
lemmatizer = WordNetLemmatizer()
def lemmatize_token(token_pos_tuple):
    if token_pos_tuple == None:
        return ""
    else:
        return lemmatizer.lemmatize(word=token_pos_tuple[0],pos=token_pos_tuple[1])

def lemmatize_text(tweet_text):
    if len(tweet_text) > 0:
        return list(map(lemmatize_token,tweet_text))
    else:
        return [""]
reduced_df["text"] = list(map(lemmatize_text,list(reduced_df["text"])))

<ipython-input-29-49446b9a00bc>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df["text"] = list(map(lemmatize_text,list(reduced_df["text"])))


In [ ]:
max_tokens = 30000
max_sequence_len = max(list(reduced_df["text"].apply(lambda x: len(x))))
reduced_rows_idx = np.argwhere(np.array(reduced_df["text"].apply(lambda x: len(x))) >= 10)
reduced_df = reduced_df.iloc[reduced_rows_idx.reshape(reduced_rows_idx.shape[0],)]
reduced_df["text"] = list(map(lambda x: " ".join(x), list(reduced_df["text"])))
with open("/content/new_preprocessed_tweets_till.pkl","wb") as file_handle:
  pickle.dump(reduced_df["text"],file_handle)



In [ ]:
!mv /content/new_preprocessed_tweets_till.pkl /content/drive/MyDrive

In [ ]:
!cp /content/drive/MyDrive/new_preprocessed_tweets_till.pkl /content

In [ ]:
import pickle
with open("/content/new_preprocessed_tweets_till.pkl","rb") as file_handle:
    x = pickle.load(file_handle)

reduced_df = pd.DataFrame(x)
reduced_df['target'] = df.iloc[list(reduced_df.index)]['target']

In [ ]:
r_text = reduced_df['text'][x.index]
r_label = reduced_df['target'][x.index]


reduced_d = pd.DataFrame({'text':r_text,'target':r_label})
filtered_reduced_data = reduced_d[reduced_d["text"] != ""]


# max(x.values, key=len)
print("maximum length of text in filtered data ",len(max(filtered_reduced_data["text"], key=len)))
print("filtered data lenght ",len(filtered_reduced_data))

maximum length of text in filtered data  181
filtered data lenght  311834


In [ ]:
max_tokens =  30000
max_sequence_len = len(max(filtered_reduced_data["text"], key=len))
vectorize_layer = TextVectorization(max_tokens=max_tokens,
                                    output_mode="int",
                                    output_sequence_length=max_sequence_len)
vectorize_layer.adapt(data=np.array(filtered_reduced_data["text"]))

# dumping vectorizer
with open("/content/token_integer_mapping.pkl","wb") as file_handle:
  pickle.dump({"config":vectorize_layer.get_config(),
               "weights":vectorize_layer.get_weights()},file_handle)

KeyboardInterrupt: ignored

In [ ]:
!mv /content/token_integer_mapping.pkl /content/drive/MyDrive

mv: cannot stat '/content/token_integer_mapping.pkl': No such file or directory


In [ ]:
!cp /content/drive/MyDrive/token_integer_mapping.pkl /content

In [ ]:
import pickle
with open("/content/token_integer_mapping.pkl","rb") as file_handle:
  vectorize_layer_attributes = pickle.load(file_handle)
loaded_vectorization_layer = TextVectorization.from_config(vectorize_layer_attributes["config"])
loaded_vectorization_layer.set_weights(vectorize_layer_attributes["weights"])

In [ ]:
def vectorize_tweet(raw_tweet):
  return loaded_vectorization_layer(raw_tweet).numpy()

# vectorizing the data/text
vectorized_tweets = list(filtered_reduced_data["text"].apply(vectorize_tweet))

# splitting into training and cross validation
vectorized_train_tweets = vectorized_tweets[0:200000]
vectorized_cv_tweets = vectorized_tweets[200000:]

np.savez_compressed("./vectorized_train_tweets.npz",*vectorized_train_tweets)
np.savez_compressed("./vectorized_cv_tweets.npz",*vectorized_cv_tweets)

In [ ]:
!mv /content/vectorized_train_tweets.npz /content/drive/MyDrive
!mv /content/vectorized_cv_tweets.npz /content/drive/MyDrive

In [ ]:
!cp /content/drive/MyDrive/vectorized_train_tweets.npz /content
!cp /content/drive/MyDrive/vectorized_cv_tweets.npz /content

In [ ]:
tweet_labels = list(filtered_reduced_data["target"].replace(to_replace=filtered_reduced_data["target"].unique(),
                                        value=list(range(len(filtered_reduced_data["target"].unique())))))

train_labels = tweet_labels[0:200000]
cv_labels = tweet_labels[200000:]



train_mb_size = 1000 # mini batch
num_epochs = 10
train_size = 200000
cv_mb_size = 1577
cv_size = 14193


loaded_vectorized_cv_tweets = np.load("./vectorized_cv_tweets.npz")
loaded_vectorized_train_tweets = np.load("./vectorized_train_tweets.npz")

In [ ]:
def train_datagen():

  for i in range( train_size//train_mb_size):

    tweets_mb_list = [loaded_vectorized_train_tweets["arr_"+str(arr_idx)] for arr_idx in range(i*train_mb_size,train_mb_size*(i+1))]
    tweets_labels_mb_list = [np.array(tweet_label) for tweet_label in train_labels[i*train_mb_size:train_mb_size*(i+1)]]

    yield np.array(tweets_mb_list), np.expand_dims(np.array(tweets_labels_mb_list),-1)

def cv_datagen():

  for i in range(cv_size//cv_mb_size):

    tweets_mb_list = [loaded_vectorized_cv_tweets["arr_"+str(arr_idx)] for arr_idx in range(i*cv_mb_size,cv_mb_size*(i+1))]
    tweets_labels_mb_list = [np.array(tweet_label) for tweet_label in cv_labels[i*cv_mb_size:cv_mb_size*(i+1)]]

    yield np.array(tweets_mb_list), np.expand_dims(np.array(tweets_labels_mb_list),-1)



In [ ]:
with open("/content/glove.6B.zip","rb") as f:
  data_of_glove = text.load(f)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
!unzip /content/glove.6B.zip

In [ ]:
max_vocabulary_size = len(loaded_vectorization_layer.get_vocabulary())
embedding_output_dim = 50
max_sequence_len = len(max(filtered_reduced_data["text"], key=len))

rnn_model = Sequential()
rnn_model.add(Input(shape=(None,), dtype="int64"))
rnn_model.add(Embedding(
                        input_dim = max_vocabulary_size,
                        output_dim =  embedding_output_dim,
                        input_length = max_sequence_len
                        # weights=[embedding_matrix],
                        # trainable=False)
))
rnn_model.add(LSTM(units=50))
rnn_model.add(Dense(units=1,activation="sigmoid"))
print(rnn_model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 50)          1500000   
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 1,520,251
Trainable params: 1,520,251
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
rnn_model.compile(loss="binary_crossentropy",metrics=["accuracy",
                                                      tf.keras.metrics.Precision(),
                                                      tf.keras.metrics.Recall()])

In [ ]:
training_data_gen = train_datagen()
cv_data_gen = cv_datagen()
num_epochs = 10
rnn_model.fit(training_data_gen,
              batch_size=train_mb_size,
              epochs=num_epochs,
              validation_data=cv_data_gen,
              validation_batch_size=cv_mb_size,
              steps_per_epoch=50)

#  !cp /content

Epoch 1/10
50/50 [==============================] - 337s 7s/step - loss: 0.0879 - accuracy: 1.0000 - precision_2: 0.0000e+00 - recall_2: 0.0000e+00 - val_loss: 5.8144 - val_accuracy: 0.0000e+00 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00
Epoch 2/10
50/50 [==============================] - ETA: 0s - loss: 0.0015 - accuracy: 1.0000 - precision_2: 0.0000e+00 - recall_2: 0.0000e+00

50/50 [==============================] - 345s 7s/step - loss: 0.0015 - accuracy: 1.0000 - precision_2: 0.0000e+00 - recall_2: 0.0000e+00
Epoch 3/10
50/50 [==============================] - 368s 7s/step - loss: 3.5334e-04 - accuracy: 1.0000 - precision_2: 0.0000e+00 - recall_2: 0.0000e+00
Epoch 4/10
50/50 [==============================] - 434s 9s/step - loss: 0.8535 - accuracy: 0.7665 - precision_2: 1.0000 - recall_2: 0.6427
Epoch 5/10


50/50 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - precision_2: 0.0000e+00 - recall_2: 0.0000e+00


In [ ]:
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(rnn_model, open(filename, 'wb'))

In [ ]:
#  !mv /content/vectorized_train_tweets.npz /content/drive/MyDrive
# some time later...

# load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.sc

# new_v = TextVectorization.from_config(d['config'])
# new_v.set_weights(d['weights'])


In [ ]:
# def vectorize_text(text):
#    text = tf.expand_dims(text,-1)
#    vectorize_layer.adapt(data=np.array(text))
#    return vectorize_layer(text)
# vector_data= list(filtered_data['Text'].apply(vectorize_text))


In [ ]:
label = list(filtered_data['target'])

with open("/content/drive/MyDrive/DataSet/vectorized_tweets_list.pkl","rb") as file:
  list_of_vector=pickle.load(file)

In [ ]:
# import os
# path_to_glove_file = os.path.join(
#     os.path.expanduser("~"), "/content/glove.6B.50d.txt"
# )
# embeddings_index = {}
# with open(path_to_glove_file) as f:
#      for line in f:
#         word, coefs = line.split(maxsplit=1)
#         coefs = np.fromstring(coefs, "f", sep=" ")
#         embeddings_index[word] = coefs

# print("Found %s word vectors." % len(embeddings_index))
# num_tokens =max_vocab_size + 2
# embedding_dim = 50
# hits = 0
# misses = 0
# embedding_matrix = np.zeros((num_tokens, embedding_dim))
# embedding_matrix.shape

Found 400000 word vectors.


(30002, 50)

In [ ]:
# voc = new_vectorizer.get_vocabulary()
# word_index = dict(zip(voc, range(len(voc))))
# for word, i in word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         # Words not found in embedding index will be all-zeros.
#         # This includes the representation for "padding" and "OOV"
#         embedding_matrix[i] = embedding_vector
#         hits += 1
#     else:
#         misses += 1
# print("Converted %d words (%d misses)" % (hits, misses))


Converted 24277 words (5723 misses)


In [ ]:
max_vocab_size = 30000+2
embedding_output_dim = 50
max_sequence_len = 256

rnn_model = Sequential()
rnn_model.add(Input(shape=(None,), dtype="int64"))
rnn_model.add(Embedding(
                        max_vocab_size,
                        embedding_output_dim ,
                        input_length=max_sequence_len,
                        weights=[embedding_matrix],
                        trainable=False)
)
rnn_model.add(LSTM(units=50))
rnn_model.add(Dense(units=1,activation="sigmoid"))
print(rnn_model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          1500100   
                                                                 
 lstm (LSTM)                 (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 1,520,351
Trainable params: 20,251
Non-trainable params: 1,500,100
_________________________________________________________________
None


In [ ]:
rnn_model.compile(
    loss="binary_crossentropy", optimizer="rmsprop", metrics=["acc"]
)

In [ ]:
# lb = pd.DataFrame({"label":label})
# lb['label'].replace({4:1},inplace=True)
# label = np.array(lb)
# list_of_vector = list_of_vector[:len(list_of_vector)-204]
# label = label[:len(label)-204]
# len(label)
batch_size = (int)(len(list_of_vector)/100)
batch_size

13960

In [ ]:
global i
def DataGen(list_of_vector,label,batch_size):
  i=0
  x=[]
  while (i+batch_size)<len(list_of_vector):
    list_of_array = [np.array(a) for a in list_of_vector[i:i+batch_size]]
    list_of_label = label[i:i+batch_size]
    i+=batch_size
    yield np.array(list_of_array),np.array(list_of_label)

In [ ]:
type((list_of_vector))

list

In [ ]:
for x,y in DataGen(list_of_vector,label,batch_size):
  print(x.shape)
  break

IndentationError: ignored

In [ ]:
for x,y in DataGen(list_of_vector,label,batch_size):
  rnn_model.fit(DataGen(list_of_vector,label,batch_size),epochs=1)

ValueError: ignored

In [ ]:
model.compile(
    loss="binary_crossentropy", optimizer="rmsprop", metrics=["acc"]
)

In [ ]:
global i
global j
def Data_Generator(list_of_vector,label,batch_size):
  i=0
  j=0
  x_train=[]
  y_train=[]
  while i<=(len(list_of_vector)) and i+batch_size<(len(list_of_vector)):
    x_train.append(np.array([np.array(ten) for ten in list_of_vector[i:batch_size+i]]))
    i+=batch_size
    y_train.append(label[j:batch_size+j])
    j+=batch_size
    yield np.concatenate(x_train,axis=0)

In [ ]:
Data_Generator(list_of_vector,label,batch_size)
#  np.concatenate(y_train,axis=0).reshape(batch_size,1)


In [ ]:
# lb=pd.DataFrame({"label":label})
for x_train in Data_Generator(list_of_vector,label,batch_size):
  print(x_train.shape)


In [ ]:
j=0
def Data_Generator_y(label,batch_size):
  y_train=[]
  while j<(len(label)):
    y_train.append(label[j:batch_size+j])
    y+=batch_size
  yield np.array(y_train)

In [ ]:
y_train = Data_Generator_y(label,13000)

In [ ]:
y_train

SyntaxError: ignored